In [ ]:
'''
This script is used for interacting with OpenLCA.
Codes are developed by Ao Chen @ETH-ML.
Connect to IPC server in OpenLCA software before running. (Port: 8080)
'''

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})

import olca
import uuid
import math
import pandas as pd
from datetime import datetime, date

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [ ]:
def get_all_process(client):
    # Get all process dataset in dataframe
    Process_descriptor = client.get_descriptors(olca.Process)
    Process_list = []
    ID_list = []
    Location_list = []

    for process in Process_descriptor:
        Process_list.append(process.name)
        ID_list.append(process.id)
        Location_list.append(process.location)
    Processes_df = pd.DataFrame(list(zip(Process_list,
                                    ID_list, Location_list)),
                                columns=['name', 'id', 'location'])
    return Processes_df

In [ ]:
Processes_df = get_all_process(client)

In [ ]:
List = [Processes_df.iloc[p] for p in range(len(Processes_df)) if 'heat production' in Processes_df.iloc[p]['name']]

In [ ]:
List[1]['id']

In [ ]:
List = [['a', 1], ['b']]

In [ ]:
Dict = {'a':{'c':[1, 3], 'd':1}, 'b':2}

In [ ]:
Dict = {}

In [ ]:
Dict{'a'} = 

In [ ]:
Dict['a']['d']

In [ ]:
flat_list = [item for sublist in List for item in sublist]

In [55]:
class LCA_Calculation():
    def __init__(self, Processes_df, Impact_method, Result_path, Excel_path='', Process_list_target=None):
        self.Processes_df = Processes_df
        self.Impact_method = Impact_method
        self.Result_path = Result_path
        self.Excel_path = Excel_path
        self.Process_list_target = Process_list_target

        if not os.path.exists(self.Result_path):
            os.makedirs(self.Result_path)
    
    def create_calc_product_system(self, search_process_list, Created_PS_refProcess):
        product_system_ids = []
        product_system_names = []
        product_system_locations = []
        # for j in range(len(search_process_list)):
        for j in range(1): # for test
            # Create product systems for selected processes
            search_process_ID = search_process_list[j]['id']
            search_process_name = search_process_list[j]['name']
            search_process_location = search_process_list[j]['location']
            if search_process_ID not in Created_PS_refProcess.keys():
                print('Creating product system of: '+search_process_name+' | '+search_process_location)
                product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
                product_system_ids.append(product_system.id)
                product_system_names.append(search_process_name)
                product_system_locations.append(search_process_location)
                Created_PS_refProcess[search_process_ID] = {'Process': [search_process_name, search_process_location, search_process_ID], 'Product_system': product_system.id}
            else:
                print('This product system has been created before.')
                product_system_ids.append(Created_PS_refProcess[search_process_ID]['Product_system'])
                product_system_names.append(Created_PS_refProcess[search_process_ID]['Process'][0])
                product_system_locations.append(Created_PS_refProcess[search_process_ID]['Process'][1])
            
            # print('Remaining number of product systems to be created: '+str(len(search_process_list)-i-1))
            print('Remaining number of product systems to be created: '+str(2-j-1)) # for test

        product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids, product_system_locations)), 
                                columns=['name', 'id', 'location'])
        
        setup = olca.CalculationSetup()
        setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
        setup.impact_method = client.find(olca.ImpactMethod, self.Impact_method) # TODO: choose impact method accordingly
        setup.amount = 1.0
        # amount is the amount of the functional unit (fu) of the system that should be used in the calculation; unit, flow property, etc. of the fu
        # can be also defined; by default openLCA will take the settings of the reference flow of the product system

        calc_result = {}
        # for j in range(len(product_system_df)):
        for j in range(1): # for test
            key = product_system_names[j]+' | '+product_system_locations[j]+' | '+product_system_ids[j]
            print('Calculating '+key)
            setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[j])
            calc_result[key] = client.calculate(setup)
            # client.excel_export(calc_result, 'calc_result/'+product_system_locations[j]+'_'+'calc_result.xlsx')
            print('Remaining number of processes to be calculated: '+str(len(product_system_names)-j-1))

        keys = list(calc_result.keys())
        result_list = []
        
        for j in range(len(calc_result)):
            key = keys[j]
            result = calc_result[key]
            for k in range(len(result.impact_results)):
                GWP_result = result.impact_results[k]
                if 'GWP 100a' in GWP_result.impact_category.name:
                    GWP100a_value = GWP_result.value
            # GWP_category = GWP100a_result.impact_category
            # if GWP_category.name != 'climate change - GWP 100a':
            #     print('ERROR: This results is recording '+GWP_category.name+'.')
            # GWP100a_value = GWP100a_result.value
            result_list.append([GWP100a_value, key])
        result_df = pd.DataFrame(result_list, columns=['GWP100a', 'key'])
        result_df[['Activity', 'Product', 'Cutoff', 'Location', 'id']] = result_df.key.str.split('|', expand=True)
        result_df = result_df.drop(columns='key')
        return result_df
        
    def cal_from_excel(self, Created_PS_refProcess={}):
        material_df = pd.read_excel(self.Excel_path, header=None)
        material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
        material_df['Activity'] = material_df['Activity'].str.strip()

        # for i in range(len(material_df)):
        for i in range(1): # for test
            search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if material_df['Activity'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
            result_df = self.create_calc_product_system(search_process_list, Created_PS_refProcess)
            result_df.to_csv(self.Result_path+'/Summary_'+material_df['Activity'][i]+'.csv', index=None)
        
        return Created_PS_refProcess

    def cal_from_list(self, Created_PS_refProcess={}):
        for i in range(len(self.Process_list_target['include'])):
            search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if self.Process_list_target['include'][i] in self.Processes_df.iloc[p]['name'] and not self.Process_list_target['not_include'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
            result_df = self.create_calc_product_system(search_process_list, Created_PS_refProcess)
            result_df.to_csv(self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'.csv', index=None)
        
        return Created_PS_refProcess

In [48]:
len(Created_PS_refProcess)

2

In [56]:
Impact_method = 'IPCC 2013 no LT'
Today = date.today().strftime("%b_%d_%Y")
Result_path = Today+'_calc_results_GWP100a_test'

Excel_path = 'List_of_Materials_ecoinvent.xlsx' # optional
Process_list_target = {
    'include': ['market for electricity, medium voltage'], # key words you want to include
    'not_include': ['municipal waste'] # key words you do not want to include
} # optional, 
  # if you want to search different activities, put your keywords accordingly in the second, third, ... 
  # position in the list and remember to include an empty element '' when there's no 'not_include' keyword.

instance1 = LCA_Calculation(Processes_df, Impact_method, Result_path, Excel_path=Excel_path, Process_list_target=Process_list_target)
Created_PS_refProcess = instance1.cal_from_excel(Created_PS_refProcess=Created_PS_refProcess) # input Excel
Created_PS_refProcess = instance1.cal_from_list(Created_PS_refProcess=Created_PS_refProcess) # input list (in fact a dict)

# TODO: change names above as you need
# if you see `ERROR 500`, Failed to call method public org.openlca.ipc.RpcResponse org.openlca.ipc.handlers.Calculator.calculate(org.openlca.ipc.RpcRequest)`,
# try to increase max. memory usage in OpenLCA then restart OpenLCA and Jupyter kernel.

This product system has been created before.
Remaining number of product systems to be created: 1
Calculating autoclaved aerated concrete block production | autoclaved aerated concrete block | Cutoff, U | CH | c1ff205d-bda1-43fa-80d6-aad4a6faaf04
Remaining number of processes to be calculated: 0
This product system has been created before.
Remaining number of product systems to be created: 1
Calculating market for electricity, medium voltage | electricity, medium voltage | Cutoff, U | OM | 2f9e0c8e-02b4-4d11-bb60-d53f425246bb
Remaining number of processes to be calculated: 0


In [38]:
result_df

,GWP100a,Activity,Product,Cutoff,Location,id
0,0.348967,autoclaved aerated concrete block production,autoclaved aerated concrete block,"Cutoff, U",CH,c1ff205d-bda1-43fa-80d6-aad4a6faaf04


In [54]:
key = list(calc_result.keys())[0]
test_result = calc_result[key]
for i in range(len(test_result.impact_results)):
    test_GWP_result = test_result.impact_results[i]
    if 'GWP 100a' in test_GWP_result.impact_category.name:
        GWP100a_value = test_GWP_result.value


4

In [23]:
test_GWP_result = test_result.impact_results[1]

In [37]:
test_GWP_result.impact_category

Ref(id='80c009aa-8eff-3a7e-b049-ec78d1d6583d', olca_type='ImpactCategory', name='climate change - GWP 100a', description=None, version=None, last_change=None, category_path=None, library=None, ref_unit='kg CO2-Eq', location=None, flow_type=None, process_type=None)

In [ ]:
test_GWP_category = test_GWP_result.impact_category

In [ ]:
test_GWP_category.name

In [ ]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []
location_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
    location_list.append(process.location)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list, location_list)),
                               columns=['name', 'id', 'location'])
# processes_df

In [ ]:
def search_process(processes_df):
    '''
    Search specific flow databases with key word(s)
    '''
    search_process_df = processes_df[processes_df['name'].str.contains(r'heat production, heavy fuel oil, at industrial(?!$)')] # TODO: change keywords
    search_process_df.reset_index(drop=True, inplace=True)
    return search_process_df

In [ ]:
test_process = client.get(olca.Process, uid='4f582f90-d0ae-3552-be43-70c730e82af0')

In [ ]:
test_exchange = test_process.exchanges[0]

In [ ]:
test_exchange.amount

In [ ]:
test_amount = []

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_exchange.amount = 0.5

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_process

In [ ]:
process_list_all[0]

In [ ]:
process_to_search_df = pd.read_excel('List_of_Materials_ecoinvent.xlsx', header=None)

In [ ]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
product_system_locations = pd.Series(search_process_df['location'])

# for i in range(len(search_list_processID)):
for i in range(2): # for test
    print('Creating product system of: '+search_process_df['name'][i]+' | '+product_system_locations[i])
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)
    # print('Remaining number of product systems to be created: '+str(len(search_list_processID)-i-1))
    print('Remaining number of product systems to be created: '+str(2-i-1)) # for test

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product_system_name', 'product_system_id'])

product_system_df.assign(product_system_location=product_system_locations)

In [ ]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013') # TODO: choose impact method accordingly

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [ ]:
# for ps in range(len(product_system_names)):
for ps in range(2): # for test
    
    print('Working on '+product_system_names[ps]+' | '+product_system_locations[ps]+' | '+product_system_ids[ps])

    setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[ps])

    print(setup.product_system.id)

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result/'+product_system_locations[ps]+'_'+'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(len(product_system_names)-ps-1))

In [ ]:
GWP100a_df = pd.DataFrame()
# for i in range(len(product_system_locations)):
for i in range(2): # for test
    filename_xlsx = 'calc_result/'+product_system_locations[i]+'_calc_result.xlsx'
    # filename_xlsx = 'calc_result/'+product_system_locations[0]+'_calc_result.xlsx' # for test
    results_db = pd.ExcelFile(filename_xlsx)
    impact_df = results_db.parse('Impacts')
    setup_df = results_db.parse('Calculation setup')

    new_header = impact_df.iloc[0] #grab the first row for the header
    impact_df = impact_df[1:] #take the data less the header row
    impact_df.columns = new_header #set the header row as the df header

    GWP100a_series = pd.Series(impact_df.iloc[2, 2:])
    GWP100a_series['Process'] = search_process_df['name'][i]
    GWP100a_series['location_extended'] = setup_df.iloc[3, 2]

    GWP100a_df[product_system_locations[i]] = GWP100a_series

GWP100a_df.T.to_excel('GWP100a_summary.xlsx')


In [ ]:
process